# Laotian Immigration Patterns in 2016
### Data Engineering Capstone Project

#### Project Summary
This project will investigate trends of Laotian immigration to the US in 2016. Raw datasets on immigration, city temperature and city demographics will be collected and uploaded into S3. Then python and Spark will be used to transform the raw data from S3 into a database schema suitable for analysis in a Redshift data warehouse. This ETL pipeline will be orchestrated using Airflow. 

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

In [2]:
# create a spack session
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 

We are interested in investigating any patterns in temperature and city demographics that is observed for Laotian immigrants. To do this, 
datasets on immigration, city temperature, and city demographics will first be collected. 

Next, a data model will be created that is optimized for joins against city data. This data model will consist of dimension tables for city dempographics and city temperatures. Fact tables will be created from the immigration dataset. 

This star schema data model will then allow for queries that analyze what type of temperature and demographics in a destination city are common for Laotian immigrants. Since the immigration dataset is large (over 1 million records), Spark will be used to allow for massively parallel data processing.

#### Describe and Gather Data 
The I94 Immigration Data is in SAS7BDAT format and comes from the US National Tourism and Trade Office: https://travel.trade.gov/research/reports/i94/historical/2016.html
The data dictionary is in the _I94_SAS_Labels_Descriptions.SAS_ file. 

##### Notable fields include: 
- i94yr - 4 digit year
- i94mon - Numeric month
- i94cit - origin
- i94port - arrival destination (city, state for US)
- arrdate - arrival Date in the USA
- i94mode - mode of transportation
- depdate - the Departure Date from the USA
- i94bir - Age of Respondent in Years
- i94visa - Visa codes (1=Business, 2=Pleasure, 3=Student)
- dtadfile - Character Date Field
- occup - Occupation that will be performed in U.S.
- biryear - 4 digit year of birth
- gender - Non-immigrant sex


The World Temperature Data is in csv format and comes from Kaggle: 
https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data

The U.S. City Demographic Data is in csv format and comes from OpenSoft: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/

The Airport Code Table is a simple csv table of airport codes and corresponding cities. It comes from here: https://datahub.io/core/airport-codes#data (_NOTE: this dataset may not be needed in our data model_)

#### Investigate the raw dataset

In [5]:
# Read in the data

# immigration data sample
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

# city temperature data 
fname2 = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(fname2)

# city demographics
df_citydemo = pd.read_csv('us-cities-demographics.csv', delimiter=';')

# airport data
df_airport = pd.read_csv('airport-codes_csv.csv')


In [6]:
#immigration data

pd.set_option('display.max_columns', 28)
print(df.shape)
df.head()

(3096313, 28)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [10]:
# immigration data sample
fname = '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat'
df2 = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
print(df2.shape)
df2.head()

(3574989, 34)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4.0,2016.0,6.0,135.0,135.0,XXX,20612.0,NaN,NaN,NaN,...,U,NaN,1957.0,10032016,NaN,NaN,NaN,1.493846e+10,NaN,WT
1,5.0,2016.0,6.0,135.0,135.0,XXX,20612.0,NaN,NaN,NaN,...,U,NaN,1966.0,10032016,NaN,NaN,NaN,1.746006e+10,NaN,WT
2,6.0,2016.0,6.0,213.0,213.0,XXX,20609.0,NaN,NaN,NaN,...,U,NaN,1989.0,D/S,NaN,NaN,NaN,1.679298e+09,NaN,F1
3,7.0,2016.0,6.0,213.0,213.0,XXX,20611.0,NaN,NaN,NaN,...,U,NaN,1993.0,D/S,NaN,NaN,NaN,1.140963e+09,NaN,F1
4,16.0,2016.0,6.0,245.0,245.0,XXX,20632.0,NaN,NaN,NaN,...,U,NaN,1992.0,D/S,NaN,NaN,NaN,1.934535e+09,NaN,F1


In [28]:
#temperature data

print(df_temp.shape)
df_temp.head()
df_temp[(df_temp['dt'] > '2010-01-01') & (df_temp['dt'] < '2017-01-01')].head()

(8599212, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
3195,2010-02-01,-2.691,0.272,Århus,Denmark,57.05N,10.33E
3196,2010-03-01,2.429,0.427,Århus,Denmark,57.05N,10.33E
3197,2010-04-01,7.123,0.234,Århus,Denmark,57.05N,10.33E
3198,2010-05-01,10.657,0.314,Århus,Denmark,57.05N,10.33E
3199,2010-06-01,14.989,0.272,Århus,Denmark,57.05N,10.33E


In [18]:
countries = df_temp['Country'].unique()
#sorted(countries, reverse=True)

In [19]:
print(df_citydemo.shape)
df_citydemo.head()

(2891, 12)


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [20]:
df_citydemo[(df_citydemo['State Code']=='CA') & (df_citydemo['City']=='San Diego')]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
236,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,White,949388
524,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Asian,267222
1392,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Hispanic or Latino,425414
1754,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Black or African-American,111650
2769,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,American Indian and Alaska Native,16496


In [8]:
print(df_airport.shape)
df_airport.head()

(55075, 12)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [9]:
#df_airport = pd.read_csv('airport-codes_csv.csv')
df_airport[(df_airport['iso_region']=='US-CA') & (df_airport['municipality'] =='Los Angeles')].head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
71,01CN,heliport,Los Angeles County Sheriff's Department Heliport,300.0,NaN,US,US-CA,Los Angeles,01CN,NaN,01CN,"-118.15399932861328, 34.03779983520508"
639,0CA0,closed,Drew Medical Center Heliport,180.0,NaN,US,US-CA,Los Angeles,NaN,NaN,NaN,"-118.241997, 33.923302"
666,0CL7,heliport,Good Samaritan Hospital Heliport,473.0,NaN,US,US-CA,Los Angeles,0CL7,NaN,0CL7,"-118.264967, 34.054901"
1459,14L,heliport,Devonshire Area Heliport,1012.0,NaN,US,US-CA,Los Angeles,14L,NaN,14L,"-118.53099822998047, 34.256900787353516"
2500,22CN,heliport,ABC-TV Heliport,422.0,NaN,US,US-CA,Los Angeles,22CN,NaN,22CN,"-118.28299713134766, 34.10329818725586"


In [29]:
# write to parquet

t1 = time.time()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
t2 = time.time()
print(t2-t1)
df_spark.write.mode("overwrite").parquet("sas_data2")
t3 = time.time()
print(t3-t2)
df_spark=spark.read.parquet("sas_data2")
t4 = time.time()
print(t4-t3)

0.08606481552124023
56.82417678833008
0.14061450958251953


In [48]:
df_spark.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1


### Step 2: Explore and Assess the Data
#### Explore the Data 

From the data exploration above, many data cleaning steps are needed to prepare the raw datasets for the data model.

First, the temperature data contains many records with _NaN_ Average temperature, which should be filtered out. Also, since we are only interested in immigration to the US around 2016, the temperature data can be filtered further to contain only countries in 'United States' and dates around 2016 (2010-2017). 

For the immigration data, since we are only interested in those from origin country Laos, we can ignore all *i94cit* fields that are not equal to the LAOS code of 203.0. 

Next, the city demographics data table will need to be normalized by removing the *Race* and *Count* columns, along with the *City*, into another table. This will allow for a unique record for each city in the main demographics table.

When querying, *i94port* from immigration dataset can then be joined to *City* from both the Temperature and Demographics dataset.

#### Cleaning Steps

In order to clean the dataset, these steps will be performed:

- Temperature data: filter for records where *Country* = 'United States', *dt* between 2010-2017, and *AverageTemperature* is not _NaN_; duplicates will be dropped

- City Demographic data: the table will need to be normalized by removing the *Race* and *Count* into a separate table; duplicates will be dropped from both tables created

- I94 Immigration data: filter out only records where *i94cit* is _203.0_ (LAOS); also june dataset has 6 extra columns, so june dataset will be skipped


#### Clean and filter the dataset

In [6]:
# clean and normalize city demographics (Race and Count are columns 10-11)

df_citydemo_filt = df_citydemo.iloc[:,:10].drop_duplicates()
df_citydemo_race = df_citydemo.iloc[:,10:].drop_duplicates()

print(len(df_citydemo))
print(len(df_citydemo_filt))
print(len(df_citydemo_race))

df_citydemo_filt.to_csv("city_demo.csv", index=False)
df_citydemo_race.to_csv("city_race.csv", index=False)


2891
596
2834


In [29]:
# clean city temperature data 

t1 = time.time()
df_temp_filt = df_temp[(df_temp['Country']=='United States') & (pd.notna(df_temp['AverageTemperature'])) 
                       & (df_temp['dt'] > '2010-01-01') & (df_temp['dt'] < '2017-01-01')].drop_duplicates()

print(len(df_temp))
print(len(df_temp_filt))
df_temp_filt.to_csv("temperature_US.csv", index=False)

t2 = time.time()
print(t2-t1)

8599212
11307
1.9743242263793945


In [3]:
# clean immigration data

import shutil
import os


def filter_i94_data(file, m):
    '''
    Input: path to I94 immigration data
    Output: Spark dataframe of I94 immigration data filtered for LAOS immigrants
    '''
    print(file)
    t1 = time.time() 
    
    # Read data into Spark
    df_i94 = spark.read.format('com.github.saurfang.sas.spark').load(file)

    # Filter for entires where i94cit is 203 (LAOS)
    df_i94_filt = df_i94.filter(df_i94.i94cit.isin(203))

  #  out_dir ='laos_i94'
 #   if m == 'jan':
 #       if os.path.exists(out_dir):
 #           shutil.rmtree(out_dir)
 #       os.makedirs(out_dir)      

  #  df_i94_filt.write.mode("append").parquet(out_dir)
  #  df_i94_filt=spark.read.parquet(out_dir)
    
    t2 = time.time()
    print(t2-t1)
    return df_i94_filt


# Run function
df = pd.DataFrame()
# skip june dataset which has extra columns
month_list = ['jan', 'feb', 'mar', 'apr', 'may', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
for m in month_list:  
    filepath = '../../data/18-83510-I94-Data-2016/i94_'+m+'16_sub.sas7bdat' 
    df_i94_filt = filter_i94_data(filepath, m)
    print(m, (df_i94_filt.count(), len(df_i94_filt.columns)))
        
    if m == 'jan':
        df = df_i94_filt.toPandas()
    else:
        df.append(df_i94_filt.toPandas(), ignore_index=True)

    #df_i94_filt.show(3)

df.to_csv("i94_laos.csv", index=False)

../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
4.167316436767578
jan (77, 28)
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
0.20766663551330566
feb (50, 28)
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
0.17574572563171387
mar (55, 28)
../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
0.2185685634613037
apr (113, 28)
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
0.17664456367492676
may (93, 28)
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
0.17773652076721191
jul (250, 28)
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
0.20240092277526855
aug (218, 28)
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
0.16498851776123047
sep (135, 28)
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
0.14185786247253418
oct (200, 28)
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
0.15395283699035645
nov (119, 28)
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat
0.16918325424194336
dec (125, 28)


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
![database schema]("i94_city_demo.png")

The data model will consist of a dimension table for the city temperature, city demographics and race demographics. Then there will be a fact table for the i94 immigration data.

This star schema data model will allow joining immigration data to features of a city that they immigrated to, such as temerature and various demographics (race/gender/population). More specifically, this data model allows queries that involve joining *i94port* from immigration dataset to *City* from both the Temperature and City Demographics datasets.

#### 3.2 Mapping Out Data Pipelines

The data pipeline to process the raw data into the data model for analysis are as follows:

##### Pre-Processing
1. Clean and filter the dataset:
   - Use python to clean and filter the temperature data to only US cities around 2016
   - ~~Use python to normalize the city demographics dataset into 2 tables: 1 normalized for unique cities, the other with Race demographics~~ (Transformaton will happen during ETL)
   - Use spark to filter the i94 immigration dataset to only immigrants from LAOS and save into parquet files
2. Upload the cleaned dataset into S3

##### ETL
1. Use SQL to create the redshift staging tables
2. Use SQL to create the redshift fact and dimension tables for the data model
3. Use the cleaned and filtered datasets in S3 as the source data to insert into the redshift staging tables
4. Transform and load the data from tge staging tables into the final redshift star schema tables 
5. Verify tables are not empty and record count in table and csv matches for those that should match

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [1]:
# clean the raw dataset
!python data_cleaning.py

In [37]:
# upload to S3
!python upload_to_S3.py

In [52]:
# run ETL to transform data from S3 into data model in redshift
# Requires going through AWS_Setup.ipynb to setup Redshift cluster first
!python etl.py

Traceback (most recent call last):
  File "etl.py", line 47, in <module>
    main()
  File "etl.py", line 41, in main
    insert_tables(cur, conn)
  File "etl.py", line 28, in insert_tables
    cur.execute(query)
psycopg2.ProgrammingError: column "dt" does not exist in s, c, unnamed_join



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.


citydemo
- citydemo_id: autoincremented primary key
- City: distinct city from us-cities-demographics.csv
- State: state of city from us-cities-demographics.csv
- MedianAge: median age of city from us-cities-demographics.csv
- MalePopulation: male population of city from us-cities-demographics.csv
- FemalePopulation: femalepopulation of city from us-cities-demographics.csv
- TotalPopulation: total population of city from us-cities-demographics.csv
- NumberVeterans: # of vertains in city from us-cities-demographics.csv
- ForeignBorn: # foreign born in city from us-cities-demographics.csv
- AverageHouseholdSize: average household size in city from us-cities-demographics.csv
- StateCode: state code of city from us-cities-demographics.csv

cityrace
- cityrace_id: autoincremented primary key
- citydemo_id: foreign key to citydemo on City
- Race: race being counted
- Count: count of race in city

temperature
- temperature_id: autoincremented primary key
- dt: date of temperature from GlobalLandTemperaturesByCity.csv
- AverageTemperature: average temperature from GlobalLandTemperaturesByCity.csv
- AverageTemperatureUncertainty: average temperature uncertainty from GlobalLandTemperaturesByCity.csv
- City: city having temperature from GlobalLandTemperaturesByCity.csv
- citydemo_id: foreign key to citydemo on City
- Country: country of city from GlobalLandTemperaturesByCity.csv
- Latitude: latitude from GlobalLandTemperaturesByCity.csv
- Longitude: longitude from GlobalLandTemperaturesByCity.csv

i94
- i94_id: autoincremented primary key
- i94yr - 4 digit year from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94mon - Numeric month from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94cit - origin location from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94port - arrival destination (city, state for US)  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- citydemo_id: foreign key to citydemo on City
- arrdate - arrival Date in the USA  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94mode - mode of transportation  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- depdate - the Departure Date from the USA  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94bir - Age of Respondent in Years  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- i94visa - Visa codes (1=Business, 2=Pleasure, 3=Student)  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- dtadfile - Character Date Field  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- occup - Occupation that will be performed in U.S.  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- biryear - 4 digit year of birth  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat
- gender - Non-immigrant sex  from 18-83510-I94-Data-2016/i94_XXX16_sub.sas7bdat


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.